In [ ]:
from aoc.day09 import IntCodeComputer
import numpy as np
import functools

In [ ]:
with open("17-input.txt", "rt") as FILE:
    data = FILE.read()
    data = data.split(",")
    data = [int(d) for d in data]
data[:8]

In [ ]:
class Map:
    heading_list = ["<","^",">","v"]*3
    heading_coords = [(-1,0),(0,-1),(1,0),(0,1)]*3

    def __init__(self):
        self.map = ""
        
    def output_value(self, value):
        self.map += chr(value)
        
    def width(self):
        return self.map.index('\n')

    def height(self):
        return int(len(self.map) / (self.width()+1))
    
    def get_value_at(self, x, y):
        if x<0 or y<0:
            return " "
        line_length = self.width()
        try:
            return self.map[((line_length+1)*y)+x]
        except IndexError:
            return " "
    
    def get_coords_at_heading(self, pos, heading, offset=0):
        import numpy as np
        ix = self.heading_list.index(heading, 2)
        coords = self.heading_coords[ix+offset]
        return np.add(pos, coords), self.heading_list[ix+offset]
        
    def get_value_at_heading(self, pos, heading, offset=0):
        coords, heading = self.get_coords_at_heading(pos, heading, offset)
        return self.get_value_at(*coords), coords, heading
        

        
scaffold_map = Map()

computer = IntCodeComputer(list(data), debug=False)
computer.output_value = scaffold_map.output_value
computer.run()

print(scaffold_map.map)

In [ ]:
alignment_params = 0
for y in range(1, scaffold_map.height()-2):
    print()
    for x in range(1, scaffold_map.width()-2):
        v = scaffold_map.get_value_at(x, y)
        if v == "#":
            n = 0
            n += 1 if scaffold_map.get_value_at(x+1, y) == '#' else 0
            n += 1 if scaffold_map.get_value_at(x-1, y) == '#' else 0
            n += 1 if scaffold_map.get_value_at(x, y+1) == '#' else 0
            n += 1 if scaffold_map.get_value_at(x, y-1) == '#' else 0
            if n == 4: 
                v = 'O'
                alignment_params += x*y
        print(v, end='')

print()        
print(alignment_params)

# Part 2

In [ ]:
import re, math, numpy as np
robot_pos = re.search('[\^v<>]', scaffold_map.map)

y = math.floor(robot_pos.start() / (scaffold_map.width()+1))
x = robot_pos.start() - y*(scaffold_map.width()+1)
pos = (x,y)
heading = scaffold_map.get_value_at(*pos)

print(pos, heading)

running = True
path = ""
while running:
    # Can we go straight
    v_s, pos_s, h_s = scaffold_map.get_value_at_heading(pos, heading)
    v_l, pos_l, h_l = scaffold_map.get_value_at_heading(pos, heading, -1)
    v_r, pos_r, h_r = scaffold_map.get_value_at_heading(pos, heading, 1)
    if v_s == "#":
        path += ("-")
        pos = pos_s
        heading = h_s
    elif v_l == '#':
        path += ("L")
        heading = h_l
    elif v_r == '#':
        path += ("R")
        heading = h_r
    else:
        running = False


abbreviated = []
for c in path:
    if c == "-":
        abbreviated[-1] = abbreviated[-1]+1
    else:
        abbreviated += [c,0]

abbreviated_string = ",".join([str(a) for a in abbreviated])
abbreviated_string

In [ ]:
def find_best_subpattern(path):
    pattern_set = set()
    for length in range(2, len(path)):
        for p in range(len(path)):
            pattern = path[p:p+length]
                
            if "A" in pattern or "B" in pattern or "C" in pattern:
                continue
                
            if pattern[0] != "L" and pattern[0] != "R":
                continue

            if pattern[-1] == "L" or pattern[-1] == "R":
                continue

            pattern = ",".join([str(p) for p in pattern])
            
            if len(pattern)>20:
                continue
            
            pattern_set.add(pattern)
            
    abbreviated_string = ",".join([str(p) for p in path])     
    pattern_scores = []
    for pattern in pattern_set:
        m = re.findall(pattern, abbreviated_string)
        p_len = re.findall(",", pattern)
        pattern_scores.append(dict(p=pattern, score=len(pattern)*len(p_len)*len(m)))

    pattern_scores.sort(key=lambda x: x["score"], reverse=True)
    return pattern_scores[0]["p"], abbreviated_string

a, remainder = find_best_subpattern(abbreviated)
remainder = remainder.replace(a, "A")
print(a, remainder)

b, remainder = find_best_subpattern(remainder.split(","))
remainder = remainder.replace(b, "B")
print(b, remainder)

c, remainder = find_best_subpattern(remainder.split(","))
remainder = remainder.replace(c, "C")
print(c, remainder)

# Part 2 - Attempt 2

That wasn't quite it - we have to replace ALL path commands - so the challenge has to be to start from the first movement and try to find as long a repeating pattern as possible.

In [ ]:
def find_best_subpattern2(path):
    import re
    program_pattern = re.compile("[ABC]")
    direction_pattern = re.compile("[LR]")
    movement_pattern = re.compile("\d+")
    
    # Remove any starting program patterns
    start = 0
    for ix, p in enumerate(path):
        m = direction_pattern.match(p)
        if m is not None:
            start=ix
            break
    
    
    pattern_set = set()
    for length in range(2, len(path)-start):
        pattern = path[start:start+length]

#         if "A" in pattern or "B" in pattern or "C" in pattern:
#             continue
        if set("ABC") in pattern:
            break

#         if pattern[-1] == "L" or pattern[-1] == "R":
#             continue
        if movement_pattern.match(str(pattern[-1])) is None:
            continue

        pattern = ",".join([str(p) for p in pattern])

        if len(pattern)>20:
            break

        pattern_set.add(pattern)

    abbreviated_string = ",".join([str(p) for p in path])     
    pattern_scores = []
    for pattern in pattern_set:
        m = re.findall(pattern, abbreviated_string)
        if len(m) > 1:
            p_len = re.findall(",", pattern)
            pattern_scores.append(dict(p=pattern, score=len(pattern)*len(p_len)*len(m)))

    pattern_scores.sort(key=lambda x: x["score"], reverse=True)
    
    print(pattern_scores)
    return pattern_scores[0]["p"], abbreviated_string

a, remainder = find_best_subpattern2(abbreviated)
remainder = remainder.replace(a, "A")
print(a, remainder)

b, remainder = find_best_subpattern2(remainder.split(","))
remainder = remainder.replace(b, "B")
print(b, remainder)

c, remainder = find_best_subpattern2(remainder.split(","))
remainder = remainder.replace(c, "C")
print(c, remainder)

In [ ]:
ord('A')

In [ ]:
# Wake up robot
move_program = list(data)
move_program[0] = 2

class RobotIO:
    
    robot_input = [
        remainder,
        a,
        b,
        c,
        "n"
    ]

    robot_input = "\n".join(robot_input) + "\n"

    def send_robot_command(self):
        value = self.robot_input[0]
        self.robot_input = self.robot_input[1:]
        print(value, end="")
        return ord(value)

    def receive_robot_output(self, value):
        if value < 128:
            print("{}".format(chr(value)), end="")
        else:
            print(value)
        return 0
    
    
io = RobotIO()

computer = IntCodeComputer(move_program, debug=False)
computer.input_value = io.send_robot_command
computer.output_value = io.receive_robot_output
computer.run()